In [0]:
import numpy as np
import pandas as pd
from collections import Counter

In [0]:
class node(object):
  def _init_(self,f,t,left,right):
    self.f = f
    self.t = t
    self.left = left
    self.right = right

In [0]:
class leaf(object):
  def _init_(self,label):
    self.label = label

In [0]:
class dtree(object):
  
  def gini_imp(self,label):
    a = np.unique(label,return_counts=True)[1]/float(len(label))
    g = 1-np.sum(a**2)
    return g

  def split(self,x,y,a):
    gini = 100000
    f = 0
    t = 0
    for i in range(x.shape[1]):
      for j in range(len(a[i])):
        c = np.array(((x[:,i]>=a[i][j])).astype(int))
        p = np.sum(c)/len(c)
        b = self.gini_imp(y[np.where(c==1)])*p + self.gini_imp(y[np.where(c==0)])*(1-p)
        if b<gini : 
          gini = b
          f = i
          t = a[i][j]
    return f,t

  def build(self,x,y,a):
    f,t = self.split(x,y,a)
    z = np.array((x[:,f]>t).astype(int))
    y_left = y[np.where(z==0)]
    y_right = y[np.where(z==1)]
    p = len(y_right)/len(y) 
    ig = self.gini_imp(y) - p*self.gini_imp(y_right) - (1-p)*self.gini_imp(y_left)
    if ig==0 :
      n = leaf()
      n.label = Counter(y).most_common(1)[0][0]
      return n
    else:
      n = node()
      n.f = f
      n.t = t
      x_left = x[np.argwhere(z-1).flatten(),:]
      x_right = x[np.argwhere(z).flatten(),:]
      n.right = self.build(x_right,y_right,a)
      n.left = self.build(x_left,y_left,a)
      return n

  def train(self,x_train,y_train): 
    X = np.array(x_train)
    y = np.array(y_train)
    n = X.shape[1]
    m = X.shape[0]
    a = []
    for i in range(n):
      c = np.unique(X[:,i])
      a.append(c)
    self.tree = self.build(X,y,a)

  def accuracy(self,y_pred,y_true):                                             
      p = 0
      m = len(y_true)
      for a in range(m):                                                        
        if y_true[a]==y_pred[a]:                
          p+=1
      return (p*100)/m

  def predict(self,x):
      X = np.array(x)
      y_pred = np.zeros((X.shape[0],1))
      for i in range(X.shape[0]):
         a = self.tree
         while isinstance(a,leaf) != True:
           f = a.f
           t = a.t
           if X[i][f] >= t: a = a.right
           else : a = a.left
         y_pred[i] = a.label
      return y_pred


In [0]:
df = pd.read_csv('/content/sample_data/california_housing_train.csv')

In [10]:
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0
...,...,...,...,...,...,...,...,...,...
16995,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0
16996,-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179,79000.0
16997,-124.30,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313,103600.0
16998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0


In [0]:
y = df['median_house_value']
X = df.drop('median_house_value',axis=1,inplace=False)

In [0]:
model = dtree()
model.train(X,y)

In [0]:
df1 = pd.read_csv('/content/sample_data/california_housing_test.csv')

In [15]:
df1

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0
...,...,...,...,...,...,...,...,...,...
2995,-119.86,34.42,23.0,1450.0,642.0,1258.0,607.0,1.1790,225000.0
2996,-118.14,34.06,27.0,5257.0,1082.0,3496.0,1036.0,3.3906,237200.0
2997,-119.70,36.30,10.0,956.0,201.0,693.0,220.0,2.2895,62000.0
2998,-117.12,34.10,40.0,96.0,14.0,46.0,14.0,3.2708,162500.0


In [0]:
y_test = df['median_house_value']
x_test = df.drop('median_house_value',axis=1,inplace=False)

In [0]:
y_pred = model.predict(x_test)

In [0]:
model.accuracy(y_test,y_pred)